In [6]:
import numpy as np
import cv2
import time
import os
import random

In [16]:
def read_vid():
    
    name = random.choice(os.listdir("Attaquant/"))
    path = "Attaquant/"+name
    cap = cv2.VideoCapture(path)
    rate = cap.get(cv2.CAP_PROP_FPS)
    if (cap.isOpened()== False): 
          print("Error opening video stream or file")
    while(cap.isOpened()):
        ret, frame = cap.read(rate)
        if ret == True:
            time.sleep(1/rate)

            cv2.imshow(name,frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break


    cap.release()
    cv2.destroyAllWindows()

In [17]:
def record_vid(name):
    file = 'Gardien/'+ name +'.mp4'
    cap = cv2.VideoCapture(0)
    t0 = time.time()
    if (cap.isOpened() == False): 
          print("Unable to read camera feed")
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv2.VideoWriter(file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    while(True):
        ret, frame = cap.read()
        if ret == True: 
                out.write(frame)

                # Display the resulting frame    
                cv2.imshow('frame',frame)

                # Press Q on keyboard to stop recording
                if cv2.waitKey(1) & 0xFF == ord('q'):
                      break

                t1 = time.time() # current time
                num_seconds = t1 - t0 # diff
                if num_seconds > 5:
                    break
        else:
            break  
    cap.release()
    out.release()
    cv2.destroyAllWindows() 

In [21]:
def analyse():
    import argparse
    import logging
    import time

    import cv2
    import numpy as np

    from tf_pose.estimator import TfPoseEstimator
    from tf_pose.networks import get_graph_path, model_wh

    logger = logging.getLogger('TfPoseEstimator-Video')
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
    ch.setFormatter(formatter)
    logger.addHandler(ch)

    fps_time = 0


    if __name__ == '__main__':
        parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
        parser.add_argument('--resize-out-ratio', type=float, default=4.0,
    help='if provided, resize heatmaps before they are post-processed. default=1.0')
        parser.add_argument('--video', type=str, default='')
        parser.add_argument('--resolution', type=str, default='432x368', help='network input resolution. default=432x368')
        parser.add_argument('--model', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin / mobilenet_v2_large / mobilenet_v2_small')
        parser.add_argument('--show-process', type=bool, default=False,
                            help='for debug purpose, if enabled, speed for inference is dropped.')
        parser.add_argument('--showBG', type=bool, default=True, help='False to show skeleton only.')
        args = parser.parse_args()

        logger.debug('initialization %s : %s' % (args.model, get_graph_path(args.model)))
        w, h = model_wh(args.resolution)
        e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))
        cap = cv2.VideoCapture("1.mp4")

        if cap.isOpened() is False:
            print("Error opening video stream or file")
        while cap.isOpened():
            ret_val, image = cap.read()

            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=args.resize_out_ratio)
            if not args.showBG:
                image = np.zeros(image.shape)
            image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

            cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.imshow('tf-pose-estimation result', image)
            fps_time = time.time()
            if cv2.waitKey(1) == 27:
                break

        cv2.destroyAllWindows()
    logger.debug('finished+')    

In [18]:
def script(iterations, name):
    path = "Gardien/" + name
    path = path + "/"
    print(path)
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
        
    for n in range(1, iterations+1):
        print("Read video :" + str(n))
        read_vid()
        print("Record video :" + str(n))
        vid = path + name + '_'+ str(n)
        print(vid)
        record_vid(vid)
    analyse()

In [25]:
analyse()

AttributeError: module 'tensorflow.python.framework.ops' has no attribute 'RegisterShape'

In [24]:
pip install slidingwindow

  Using cached slidingwindow-0.0.14-py3-none-any.whl (9.0 kB)
Note: you may need to restart the kernel to use updated packages.
